# biomass_proportion_tern_field_data_clean_v7

Extracts STBA from the TERN basal_area_data_Grant_SEPT2022.xlsx excel spreadsheet.
The notebook joins location data with STBA proportions and applies the allometric cofactors from Cook et al. 2005) to calculate Carbon extent then applies the secondary cofactor to convert C extent to AGB extent.

The following conditions apply:

 - env = base
 - run on TERN site basal proportion data (e.g. TERN basal_area_data_Grant_SEPT2022.xlsx)

In [1]:
!conda info


     active environment : biomass_zonal
    active env location : C:\Users\robot\anaconda3\envs\biomass_zonal
            shell level : 2
       user config file : C:\Users\robot\.condarc
 populated config files : C:\Users\robot\.condarc
          conda version : 23.7.4
    conda-build version : 3.26.1
         python version : 3.11.5.final.0
       virtual packages : __archspec=1=x86_64
                          __cuda=12.6=0
                          __win=0=0
       base environment : C:\Users\robot\anaconda3  (writable)
      conda av data dir : C:\Users\robot\anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://rep

In [2]:
import pandas as pd
import geopandas as gpd
import os

In [3]:
date = "20250122"
drive_in = "E"
drive = "E"

In [4]:
# dir_ = r"{0}:\cdu\data\tern_data".format(drive_in)
# dir_ = r"X:\PGB\RSU\biomass\field_sites\tern_data"
# #output_dir = r"{0}:\cdu\data\output\{1}".format(drive, date)

In [5]:
# # dir_ = r"{0}:\cdu\data\slats\varified_by_rm".format(drive)
# # output_dir = r"{0}:\cdu\data\output\{1}".format(drive, date)
# output_dir = r"X:\PGB\RSU\biomass\agb\{0}".format(date)
# output_dir_ext = r"{0}:\biomass\agb\{1}".format(drive, date)
# #dir_ = r"X:\PGB\RSU\biomass\field_sites\slats\including_2023"

## Lenovo

In [6]:
dir_ = r"{0}:\cdu\data\tern_data".format(drive_in)
dir_ = r"C:\Users\robot\projects\biomass\field_sites\tern_data"
#output_dir = r"{0}:\cdu\data\output\{1}".format(drive, date)

In [7]:
# dir_ = r"{0}:\cdu\data\slats\varified_by_rm".format(drive)
# output_dir = r"{0}:\cdu\data\output\{1}".format(drive, date)
output_dir = r"C:\Users\robot\projects\biomass\agb\{0}".format(date)
output_dir_ext = r"{0}:\biomass\agb\{1}".format(drive, date)
#dir_ = r"X:\PGB\RSU\biomass\field_sites\slats\including_2023"

In [8]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
if not os.path.isdir(output_dir_ext):
    os.mkdir(output_dir_ext)

In [9]:
os.listdir(dir_)

['Copy of TERN basal_area_data_Grant_SEPT2022.xlsx',
 'TERN basal_area_data_Grant_SEPT2022.cpg',
 'TERN basal_area_data_Grant_SEPT2022.dbf',
 'TERN basal_area_data_Grant_SEPT2022.prj',
 'TERN basal_area_data_Grant_SEPT2022.qmd',
 'TERN basal_area_data_Grant_SEPT2022.shp',
 'TERN basal_area_data_Grant_SEPT2022.shx',
 'TERN basal_area_data_Grant_SEPT2022.xlsx']

In [10]:
# original filed data test (13 sites)
#csv_ = os.path.join(dir_, "tree_biomass_field_data_copy.csv")
# merged csv abova and obs sheet 2013
csv_ = os.path.join(dir_, "TERN basal_area_data_Grant_SEPT2022.xlsx")

In [11]:
#csv_ = "\\pgb-bas01\DENR_Satellite_Imagery$\Scratch\Rob\tern\tree_biomass_field_data\biomass_carbon\tree_biomass_field_data_copy.csv"

In [12]:
#df = pd.read_excel(csv_, sheet_name="tree_biomass_field_data", encoding='windows-1252')
df_loc = pd.read_excel(csv_, sheet_name="Site locations_NT north")
df_species = pd.read_excel(csv_, sheet_name="df_basal_area_spp_ALL")

In [13]:
df_loc.shape

(63, 9)

In [14]:
df_species.shape

(1730, 5)

In [15]:
df_loc.columns

Index(['siteName', 'siteVisitName', 'siteType', 'siteVisitStartDate',
       'site.description', 'latitude', 'longitude', 'pointPositionWithinPlot',
       'sitevisit.description'],
      dtype='object')

In [16]:
def clean_dfs(df_loc, df_species):
    # select and rename required columns for df_loc
    df_loc_ = df_loc[['siteName', 'siteVisitName', 'latitude', 'longitude']]
    df_loc_.columns = ['site_name', 'visit_date', 'latitude', 'longitude']
    
    # select and rename required columns for df species
    df_species_ = df_species[['site_name', 'standardised_name', 'basal_area_m2_ha']]
    df_species_.columns = ['site_name', 'species', 'ba_m2ha']
    
    df_all = pd.merge(df_loc_, df_species_, how='left', on=['site_name', 'site_name'])
    
    return df_all, df_loc_, df_species_

In [17]:
df_all, df_loc_, df_species_ = clean_dfs(df_loc, df_species)

In [18]:
site_loc = df_loc_.site_name.unique().tolist()
print(len(site_loc))

63


In [19]:
site_loc = df_species_.site_name.unique().tolist()
print(len(site_loc))

394


In [20]:
df_all.shape

(292, 6)

In [21]:
df_all_sites = df_all.site_name.unique().tolist()
print(len(df_all_sites))

63


In [22]:
def init_clean(df):
    
    """ This function cleans the file by dropping null easting values, 
    filling in the uid column based on the index and fills any missing datum value with 
    WGS84 (data with missing values was collected in 2013)."""
    
    # fill in uid column
    df['uid'] = df.index + 1

    # fill in datum into to GDA94
    df['datum'] = 'gda94'
    
    return df
     

In [23]:
clean_df = init_clean(df_all)

In [24]:
clean_df

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,2,gda94
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,3,gda94
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,4,gda94
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,5,gda94
...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,288,gda94
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,289,gda94
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,290,gda94
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,291,gda94


In [25]:
def convert_to_gdf(clean_df):
    gdf = gpd.GeoDataFrame(clean_df, geometry=gpd.points_from_xy(clean_df.longitude, clean_df.latitude))
    gdf = gdf.set_crs('epsg:4283')
    print(gdf.crs)
    
    return gdf

In [26]:
clean_gdf = convert_to_gdf(clean_df)

epsg:4283


In [27]:
clean_gdf

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94,POINT (132.27010 -13.55729)
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,2,gda94,POINT (132.27010 -13.55729)
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,3,gda94,POINT (132.27010 -13.55729)
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,4,gda94,POINT (132.27010 -13.55729)
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,5,gda94,POINT (132.27010 -13.55729)
...,...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,288,gda94,POINT (133.27120 -16.63024)
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,289,gda94,POINT (133.27120 -16.63024)
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,290,gda94,POINT (133.27120 -16.63024)
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,291,gda94,POINT (133.27120 -16.63024)


In [28]:
# fill null values for species and basal where no basal was recorded
clean_gdf.species.fillna("No basal", inplace = True)
clean_gdf.ba_m2ha.fillna(0, inplace=True)

In [29]:
clean_gdf

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94,POINT (132.27010 -13.55729)
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,2,gda94,POINT (132.27010 -13.55729)
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,3,gda94,POINT (132.27010 -13.55729)
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,4,gda94,POINT (132.27010 -13.55729)
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,5,gda94,POINT (132.27010 -13.55729)
...,...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,288,gda94,POINT (133.27120 -16.63024)
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,289,gda94,POINT (133.27120 -16.63024)
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,290,gda94,POINT (133.27120 -16.63024)
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,291,gda94,POINT (133.27120 -16.63024)


##### Export to csv

In [30]:
clean_gdf.to_csv(os.path.join(output_dir, "clean_gdf2.csv"))
clean_gdf.to_csv(os.path.join(output_dir_ext, "clean_gdf2.csv"))

In [31]:
clean_df_ = pd.DataFrame(clean_gdf)

In [32]:
clean_df_

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94,POINT (132.27010 -13.55729)
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,2,gda94,POINT (132.27010 -13.55729)
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,3,gda94,POINT (132.27010 -13.55729)
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,4,gda94,POINT (132.27010 -13.55729)
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,5,gda94,POINT (132.27010 -13.55729)
...,...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,288,gda94,POINT (133.27120 -16.63024)
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,289,gda94,POINT (133.27120 -16.63024)
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,290,gda94,POINT (133.27120 -16.63024)
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,291,gda94,POINT (133.27120 -16.63024)


In [33]:
clean_df_.to_csv(os.path.join(output_dir, "initial_tern_biomass_cleaned.csv"), index=False)
clean_df_.to_csv(os.path.join(output_dir_ext, "initial_tern_biomass_cleaned.csv"), index=False)

In [34]:
def search_for_names(gdf, var_):
    """ This function returns a list of column names that includes the substring (var_) """
    var_cols = [col for col in gdf.columns if var_ in col]

    return var_cols

In [35]:
clean_gdf

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94,POINT (132.27010 -13.55729)
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,2,gda94,POINT (132.27010 -13.55729)
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,3,gda94,POINT (132.27010 -13.55729)
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,4,gda94,POINT (132.27010 -13.55729)
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,5,gda94,POINT (132.27010 -13.55729)
...,...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,288,gda94,POINT (133.27120 -16.63024)
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,289,gda94,POINT (133.27120 -16.63024)
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,290,gda94,POINT (133.27120 -16.63024)
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,291,gda94,POINT (133.27120 -16.63024)


In [36]:
# var_cols =search_for_names(clean_gdf, "sp")

In [37]:
# define a list of column header basal positions
pos_list = ["cent", "north", "south", "ne", "se", "nw", "sw"]

In [38]:
# def add_columns(df2, row):
    
#     """ This function transfers iterrow data to output df. """
    
#     df2['site'] = row['site']
#     df2['uid'] =  row['uid']
#     df2['date'] = str(row['am_date'])
#     df2['factor'] = row['factor']
#     df2['loc_count'] = row['count']
    
# #     # replace 9999.0 from factor
# #     fact_ = row['factor']
    
    
# #     if fact_ == 9999.0:
# #         print("fact_: ", fact_)
# #         factor_ = 0.0
# #         print("factor_: ", factor_)  
# #     else:
# #         factor_ == fact_
      
# #     df2['factor'] = factor_
# #     df2['factor'] = row['factor']
#     df2['geometry'] = row['geometry']
#     df2['lon_gda94'] = row['lon_gda94']
#     df2['lat_gda94'] = row['lat_gda94']
    
#     return df2

In [39]:
clean_gdf.columns
clean_gdf.shape

(292, 9)

In [40]:
clean_gdf

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94,POINT (132.27010 -13.55729)
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,2,gda94,POINT (132.27010 -13.55729)
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,3,gda94,POINT (132.27010 -13.55729)
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,4,gda94,POINT (132.27010 -13.55729)
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,5,gda94,POINT (132.27010 -13.55729)
...,...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,288,gda94,POINT (133.27120 -16.63024)
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,289,gda94,POINT (133.27120 -16.63024)
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,290,gda94,POINT (133.27120 -16.63024)
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,291,gda94,POINT (133.27120 -16.63024)


In [41]:
# import numpy as np
# clean_gdf.fillna(9999, inplace=True)

In [42]:
def prop(df):
    
    """ This function calculated site proportions and returns df with alive proportions 
    ‘alv_prop’, dead proportions ‘ded_prop’ and total proportions ‘total_prop’. """
    
    print("--------------------------------------PROPORTIONS-----------------------------------")
    print(df)
    al_prop = []
    ded_prop = []
    total_prop = []
    tot_alive = df.alive.sum()
    print("tot_alive: ", tot_alive)

    tot_dead = df.dead.sum()
    print("tot_dead: ", tot_dead)
    tot_all = tot_alive + tot_dead
    print("tot_all: ", tot_all)
    for index, row in df.iterrows():
        print("-"*50)
        print("alive proportion: ", (row["alive"]/tot_alive)*100)
        al_prop.append((row["alive"]/tot_alive)*100)
        print("dead proportion: ", (row["dead"]/tot_dead)*100)
        ded_prop.append((row["dead"]/tot_dead)*100)
        print("total proportion: ", ((row["alive"] + row["dead"])/(tot_alive + tot_dead)*100))
        
        total_prop.append((row["alive"] + row["dead"])/(tot_alive + tot_dead)*100)
        
    df["alv_prop"] = al_prop
    df["ded_prop"] = ded_prop
    df["total_prop"] = total_prop
    
    return df
        

In [43]:
def define_class(df):
    
    class_ = []
    leaves = []
    leaves_oth = []
    twigs = []
    bark = []
    bark_oth = []
    wood = []
    wood_oth = []
    branches = []
    branches_oth = []
    stems = []
    stems_oth = []
    agb = []
    agb_oth = []
    root = []
    root_oth = []
    tot_bm = []
    
    
    for index, row in df.iterrows():
        
        species = row["species"]
        species = species.strip()
        #print(species)
        
    
        if species == "E. tetrodonta" or species == " Eucalyptus tetrodonta":
            
            #print("Eute")
            class_.append("Eute")
            leaves.append(122)
            leaves_oth.append(0.84)
            twigs.append(127)
            bark.append(341)
            bark_oth.append(0.99)
            wood.append(2161)
            wood_oth.append(0.93)
            branches.append(779) # was 799
            branches_oth.append(0.85)
            stems.append(2502)
            stems_oth.append(0.95)
            agb.append(3403)
            agb_oth.append(0.97)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(3945)
            
        elif species == "E. miniata" or species == "Eucalyptus miniata":
   
            #print("Eumi")
            class_.append("Eumi")
            leaves.append(50)
            leaves_oth.append(0.96)
            twigs.append(52)
            bark.append(218)
            bark_oth.append(0.92)
            wood.append(1829)
            wood_oth.append(0.95)
            branches.append(375)
            branches_oth.append(0.79)
            stems.append(2047)
            stems_oth.append(0.96)
            agb.append(2472)
            agb_oth.append(0.96)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(3014)

        elif species == "E. porrecta" or species == "Eucalyptus porrecta" or species =="C. porrecta" or species == "Corymbia porrecta":
               
            #print("Eupo")    
            class_.append("Eupo")
            leaves.append(73)
            leaves_oth.append(0.85)
            twigs.append(76)
            bark.append(326)
            bark_oth.append(0.98)
            wood.append(1289)
            wood_oth.append(0.98)
            branches.append(619)
            branches_oth.append(0.96)
            stems.append(1616)
            stems_oth.append(0.98) # was 0.90
            agb.append(2308)
            agb_oth.append(0.98)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(2850)
            
        elif species == "E. bleeseri" or species == "Eucalyptus bleeseri" or species =="C. bleeseri" or species == "Corymbia bleeseri":
            
            #print("Eubl")
            class_.append("Eubl")
            leaves.append(49)
            leaves_oth.append(0.80)
            twigs.append(51)
            bark.append(347)
            bark_oth.append(0.97)
            wood.append(2225)
            wood_oth.append(0.97)
            branches.append(1163)
            branches_oth.append(0.90)
            stems.append(2573)
            stems_oth.append(0.97)
            agb.append(3785)
            agb_oth.append(0.96)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(4327)
            
            
        elif species == "E. chlorostachys" or species == "Erythrophleum chlorostachys":
            
            #print("Erch")
            class_.append("Erch")
            leaves.append(154)
            leaves_oth.append(0.75)
            twigs.append(160)
            bark.append(401)
            bark_oth.append(0.95)
            wood.append(1044)
            wood_oth.append(0.92)
            branches.append(814)
            branches_oth.append(0.62)
            stems.append(1445)
            stems_oth.append(0.93)
            agb.append(2413)
            agb_oth.append(0.82)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(2955)
            
        elif species == "No basal":
            
            #print("Erch")
            class_.append("Nobsl")
            leaves.append(0.0)
            leaves_oth.append(0.0)
            twigs.append(0.0)
            bark.append(0.0)
            bark_oth.append(0.0)
            wood.append(0.0)
            wood_oth.append(0.0)
            branches.append(0.0)
            branches_oth.append(0.0)
            stems.append(0.0)
            stems_oth.append(0.0)
            agb.append(0.0)
            agb_oth.append(0.0)
            root.append(0.0)
            root_oth.append(0.0)
            tot_bm.append(0.0)
            
            
        else:

            class_.append("Tefe")
            leaves.append(93)
            leaves_oth.append(0.89)
            twigs.append(97)
            bark.append(379)
            bark_oth.append(0.92)
            wood.append(1233)
            wood_oth.append(0.92)
            branches.append(935)
            branches_oth.append(0.83)
            stems.append(1612)
            stems_oth.append(0.92)
            agb.append(2640)
            agb_oth.append(0.91)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(3182)
            
    df["co_ef_nme"] = class_
    df["leaves"] = leaves
    df["leaves_r2"] = leaves_oth
    df["twigs"] = twigs
    df["bark"] = bark
    df["bark_r2"] = bark_oth
    df["wood"] = wood
    df["wood_r2"] = wood_oth
    df["branches"]= branches
    df["branches_r2"] = branches_oth
    df["stems"] = stems
    df["stems_r2"] = stems_oth
    df["agb"] = agb
    df["agb_r2"] = agb_oth
    df["root"] = root
    df["root_r2"] = root_oth
    df["tot_bm"] = tot_bm
    
    
    return df
            
    

In [44]:
# df3 = clean_gdf

In [45]:
# df3

In [46]:
df4 = define_class(clean_gdf)
df4.to_csv(os.path.join(output_dir, "tern_df.csv"), index=False)
df4.to_csv(os.path.join(output_dir_ext, "tern_df.csv"), index=False)
df4

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry,co_ef_nme,...,wood_r2,branches,branches_r2,stems,stems_r2,agb,agb_r2,root,root_r2,tot_bm
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94,POINT (132.27010 -13.55729),Tefe,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,2,gda94,POINT (132.27010 -13.55729),Tefe,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,3,gda94,POINT (132.27010 -13.55729),Erch,...,0.92,814.0,0.62,1445.0,0.93,2413.0,0.82,542.0,0.57,2955.0
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,4,gda94,POINT (132.27010 -13.55729),Tefe,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,5,gda94,POINT (132.27010 -13.55729),Tefe,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,288,gda94,POINT (133.27120 -16.63024),Tefe,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,289,gda94,POINT (133.27120 -16.63024),Tefe,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,290,gda94,POINT (133.27120 -16.63024),Tefe,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,291,gda94,POINT (133.27120 -16.63024),Tefe,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0


In [47]:
for i in df4.columns:
    print(i)

site_name
visit_date
latitude
longitude
species
ba_m2ha
uid
datum
geometry
co_ef_nme
leaves
leaves_r2
twigs
bark
bark_r2
wood
wood_r2
branches
branches_r2
stems
stems_r2
agb
agb_r2
root
root_r2
tot_bm


In [48]:
def total_ba_m2(df):
    
    # Total basal area
    tot_ba = []
    al_ba = []
    ded_ba = []
    
    # Carbom stock
    c_leaves = []
    c_twigs = []
    c_bark = []
    c_wood = []
    c_branches = []
    c_stems = []
    c_agb = []
    c_roots = []
    

        
    for index, row in df.iterrows():
        
        c_leaves.append(row["ba_m2ha"] * row["leaves"])
        c_twigs.append(row["ba_m2ha"] * row["twigs"])
        c_bark.append(row["ba_m2ha"] * row["bark"])
        c_wood.append(row["ba_m2ha"] * row["wood"])
        c_branches.append(row["ba_m2ha"] * row["branches"])
        c_stems.append(row["ba_m2ha"] * row["stems"])
        c_agb.append(row["ba_m2ha"] * row["agb"])
        c_roots.append(row["ba_m2ha"] * row["root"])
        

    # Carbon stock
    df["c_leaves"] = c_leaves
    df["c_twigs"] = c_twigs
    df["c_bark"] = c_bark
    df["c_wood"] = c_wood
    df["c_branches"] = c_branches
    df["c_stems"] = c_stems
    df["c_agb"] = c_agb
    df["c_roots"] = c_roots
            
    return df
        

In [49]:
df4.columns

Index(['site_name', 'visit_date', 'latitude', 'longitude', 'species',
       'ba_m2ha', 'uid', 'datum', 'geometry', 'co_ef_nme', 'leaves',
       'leaves_r2', 'twigs', 'bark', 'bark_r2', 'wood', 'wood_r2', 'branches',
       'branches_r2', 'stems', 'stems_r2', 'agb', 'agb_r2', 'root', 'root_r2',
       'tot_bm'],
      dtype='object')

In [50]:
df5 =total_ba_m2(df4)

In [51]:
df5

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry,co_ef_nme,...,root_r2,tot_bm,c_leaves,c_twigs,c_bark,c_wood,c_branches,c_stems,c_agb,c_roots
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94,POINT (132.27010 -13.55729),Tefe,...,0.57,3182.0,97.650000,101.850000,397.950000,1294.6500,981.750000,1692.600000,2772.000000,569.100000
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,2,gda94,POINT (132.27010 -13.55729),Tefe,...,0.57,3182.0,54.250000,56.583333,221.083333,719.2500,545.416667,940.333333,1540.000000,316.166667
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,3,gda94,POINT (132.27010 -13.55729),Erch,...,0.57,2955.0,333.666667,346.666667,868.833333,2262.0000,1763.666667,3130.833333,5228.166667,1174.333333
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,4,gda94,POINT (132.27010 -13.55729),Tefe,...,0.57,3182.0,1379.500000,1438.833333,5621.833333,18289.5000,13869.166667,23911.333333,39160.000000,8039.666667
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,5,gda94,POINT (132.27010 -13.55729),Tefe,...,0.57,3182.0,18.600000,19.400000,75.800000,246.6000,187.000000,322.400000,528.000000,108.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,288,gda94,POINT (133.27120 -16.63024),Tefe,...,0.57,3182.0,9.300000,9.700000,37.900000,123.3000,93.500000,161.200000,264.000000,54.200000
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,289,gda94,POINT (133.27120 -16.63024),Tefe,...,0.57,3182.0,296.437500,309.187500,1208.062500,3930.1875,2980.312500,5138.250000,8415.000000,1727.625000
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,290,gda94,POINT (133.27120 -16.63024),Tefe,...,0.57,3182.0,9.300000,9.700000,37.900000,123.3000,93.500000,161.200000,264.000000,54.200000
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,291,gda94,POINT (133.27120 -16.63024),Tefe,...,0.57,3182.0,18.600000,19.400000,75.800000,246.6000,187.000000,322.400000,528.000000,108.400000


In [52]:
site_df_list = []
x = 1
for i in df5.site_name.unique():
    print(i)
    site_df = df5[df5["site_name"]== i ]
    site_df["uid"] = x
    site_df_list.append(site_df)
    x += 1
    print(x)
    #print(site_df)
    #df5["uid"] = 
df6 = pd.concat(site_df_list)

NTAARP0001
2
NTAARP0002
3
NTAARP0003
4
NTADAC0001
5
NTADAC0002
6
NTAGFU0001
7
NTAGFU0002
8
NTAGFU0003
9
NTAGFU0004
10
NTAGFU0005
11
NTAGFU0006
12
NTAGFU0007
13
NTAGFU0008
14
NTAGFU0009
15
NTAGFU0010
16
NTAGFU0011
17
NTAGFU0012
18
NTAGFU0013
19
NTAGFU0014
20
NTAGFU0015
21
NTAGFU0016
22
NTAGFU0017
23
NTAGFU0018
24
NTAGFU0019
25
NTAGFU0020
26
NTAGFU0021
27
NTAGFU0022
28
NTAGFU0023
29
NTAGFU0024
30
NTAGFU0025
31
NTAGFU0026
32
NTAGFU0027
33
NTAGFU0028
34
NTAGFU0029
35
NTAGFU0030
36
NTAGFU0031
37
NTAGFU0032
38
NTAGFU0033
39
NTAGFU0034
40
NTAGFU0035
41
NTAGFU0036
42
NTAGFU0037
43
NTAGFU0038
44
NTAGFU0039
45
NTAGFU0040
46
NTAGFU0041
47
NTAMGD0001
48
NTAMGD0002
49
NTAMGD0003
50
NTAMGD0004
51
NTAMGD0005
52
NTAPCK0001
53
NTAPCK0002
54
NTASTU0001
55
NTASTU0002
56
NTASTU0003
57
NTASTU0004
58
NTASTU0005
59
NTTDAB0001
60
NTTDAC0001
61
NTTMGD0001
62
NTTPCK0001
63
NTTSTU0001
64


C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [53]:
df6

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry,co_ef_nme,...,root_r2,tot_bm,c_leaves,c_twigs,c_bark,c_wood,c_branches,c_stems,c_agb,c_roots
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94,POINT (132.27010 -13.55729),Tefe,...,0.57,3182.0,97.650000,101.850000,397.950000,1294.6500,981.750000,1692.600000,2772.000000,569.100000
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,1,gda94,POINT (132.27010 -13.55729),Tefe,...,0.57,3182.0,54.250000,56.583333,221.083333,719.2500,545.416667,940.333333,1540.000000,316.166667
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,1,gda94,POINT (132.27010 -13.55729),Erch,...,0.57,2955.0,333.666667,346.666667,868.833333,2262.0000,1763.666667,3130.833333,5228.166667,1174.333333
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,1,gda94,POINT (132.27010 -13.55729),Tefe,...,0.57,3182.0,1379.500000,1438.833333,5621.833333,18289.5000,13869.166667,23911.333333,39160.000000,8039.666667
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,1,gda94,POINT (132.27010 -13.55729),Tefe,...,0.57,3182.0,18.600000,19.400000,75.800000,246.6000,187.000000,322.400000,528.000000,108.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,63,gda94,POINT (133.27120 -16.63024),Tefe,...,0.57,3182.0,9.300000,9.700000,37.900000,123.3000,93.500000,161.200000,264.000000,54.200000
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,63,gda94,POINT (133.27120 -16.63024),Tefe,...,0.57,3182.0,296.437500,309.187500,1208.062500,3930.1875,2980.312500,5138.250000,8415.000000,1727.625000
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,63,gda94,POINT (133.27120 -16.63024),Tefe,...,0.57,3182.0,9.300000,9.700000,37.900000,123.3000,93.500000,161.200000,264.000000,54.200000
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,63,gda94,POINT (133.27120 -16.63024),Tefe,...,0.57,3182.0,18.600000,19.400000,75.800000,246.6000,187.000000,322.400000,528.000000,108.400000


In [54]:
df6.columns

Index(['site_name', 'visit_date', 'latitude', 'longitude', 'species',
       'ba_m2ha', 'uid', 'datum', 'geometry', 'co_ef_nme', 'leaves',
       'leaves_r2', 'twigs', 'bark', 'bark_r2', 'wood', 'wood_r2', 'branches',
       'branches_r2', 'stems', 'stems_r2', 'agb', 'agb_r2', 'root', 'root_r2',
       'tot_bm', 'c_leaves', 'c_twigs', 'c_bark', 'c_wood', 'c_branches',
       'c_stems', 'c_agb', 'c_roots'],
      dtype='object')

In [55]:
df6.to_csv(os.path.join(output_dir, "tern_df6.csv"), index=False)
df6.to_csv(os.path.join(output_dir_ext, "tern_df6.csv"), index=False)

In [56]:
df6.sample(2)

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry,co_ef_nme,...,root_r2,tot_bm,c_leaves,c_twigs,c_bark,c_wood,c_branches,c_stems,c_agb,c_roots
147,NTAGFU0031,20120706,-14.12586,134.3867,Corymbia dichromophloia,0.385714,36,gda94,POINT (134.38670 -14.12586),Tefe,...,0.57,3182.0,35.871429,37.414286,146.185714,475.585714,360.642857,621.771429,1018.285714,209.057143
134,NTAGFU0029,20120615,-15.53810,135.4090,Grevillea striata,0.375000,34,gda94,POINT (135.40900 -15.53810),Tefe,...,0.57,3182.0,34.875000,36.375000,142.125000,462.375000,350.625000,604.500000,990.000000,203.250000


In [57]:
def total_kg_per_site(df):
    df_list = []

    for uid in df.uid.unique():
        print(uid)
        df1 = df[df["uid"]==uid]
        print(df1)
        # -------------------------------- Biomass -----------------------------------
        df1["bio_l_kg1ha"] = (df1.loc[:, "c_leaves"].sum())/0.47
        
        df1["bio_t_kg1ha"] = (df1.loc[:, "c_twigs"].sum())/0.49
        
        df1["bio_b_kg1ha"] = (df1.loc[:, "c_bark"].sum())/0.49
        
        df1["bio_w_kg1ha"] = (df1.loc[:, "c_wood"].sum())/0.49
        
        df1["bio_br_kg1ha"] = (df1.loc[:, "c_branches"].sum())/0.49
        
        df1["bio_s_kg1ha"] = (df1.loc[:, "c_stems"].sum())/0.49
        
        df1["bio_r_kg1ha"] = (df1.loc[:, "c_roots"].sum())/0.49
        
        df1["bio_agb_kg1ha"] = (df1.loc[:, "c_agb"].sum())/0.49
        
        # ------------------------------ Carbon ------------------------------------
        
        df1["c_l_kg1ha"] = (df1.loc[:, "c_leaves"].sum())
        
        df1["c_t_kg1ha"] = (df1.loc[:, "c_twigs"].sum())
        
        df1["c_b_kg1ha"] = (df1.loc[:, "c_bark"].sum())
        
        df1["c_w_kg1ha"] = (df1.loc[:, "c_wood"].sum())
        
        df1["c_br_kg1ha"] = (df1.loc[:, "c_branches"].sum())
        
        df1["c_s_kg1ha"] = (df1.loc[:, "c_stems"].sum())

        df1["c_r_kg1ha"] = (df1.loc[:, "c_roots"].sum())
        
        df1["c_agb_kg1ha"] = (df1.loc[:, "c_agb"].sum())
        
        print(df1)
        df_list.append(df1)

        
        #dfmi.loc[:, ('one', 'second')]
        
#     for index, row in df.iterrows():
#         tot_ba_ha.append((row["tot_prop"] * row["ba_all_m2"])/100)
#         al_ba_ha.append((row["al_prop"] * row["ba_alive_m2"])/100)
        
#     df["alv_ba_ha"] = al_ba_ha
#     df["tot_ba_ha"] = tot_ba_ha
    final_df = pd.concat(df_list)        
    return final_df

In [58]:
df7 = total_kg_per_site(df6)

1
    site_name  visit_date  latitude  longitude                      species  \
0  NTAARP0001    20160602 -13.55729   132.2701                Acacia mimula   
1  NTAARP0001    20160602 -13.55729   132.2701           Buchanania obovata   
2  NTAARP0001    20160602 -13.55729   132.2701  Erythrophleum chlorostachys   
3  NTAARP0001    20160602 -13.55729   132.2701        Eucalyptus tetrodonta   
4  NTAARP0001    20160602 -13.55729   132.2701             Owenia vernicosa   
5  NTAARP0001    20160602 -13.55729   132.2701     Terminalia ferdinandiana   

     ba_m2ha  uid  datum                     geometry co_ef_nme  ...  root_r2  \
0   1.050000    1  gda94  POINT (132.27010 -13.55729)      Tefe  ...     0.57   
1   0.583333    1  gda94  POINT (132.27010 -13.55729)      Tefe  ...     0.57   
2   2.166667    1  gda94  POINT (132.27010 -13.55729)      Erch  ...     0.57   
3  14.833333    1  gda94  POINT (132.27010 -13.55729)      Tefe  ...     0.57   
4   0.200000    1  gda94  POINT (132.27

C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is tryin

     site_name  visit_date  latitude  longitude                 species  \
58  NTAGFU0012    20120511 -17.89564    137.019  Eucalyptus leucophloia   

     ba_m2ha  uid  datum                     geometry co_ef_nme  ...  \
58  1.388889   17  gda94  POINT (137.01900 -17.89564)      Tefe  ...   

    bio_r_kg1ha  bio_agb_kg1ha   c_l_kg1ha   c_t_kg1ha   c_b_kg1ha  c_w_kg1ha  \
58  1536.281179    7482.993197  129.166667  134.722222  526.388889     1712.5   

     c_br_kg1ha    c_s_kg1ha   c_r_kg1ha  c_agb_kg1ha  
58  1298.611111  2238.888889  752.777778  3666.666667  

[1 rows x 50 columns]
18
     site_name  visit_date  latitude  longitude                  species  \
59  NTAGFU0013    20120512 -17.89591   137.0116  Eucalyptus chlorophylla   
60  NTAGFU0013    20120512 -17.89591   137.0116   Eucalyptus leucophloia   
61  NTAGFU0013    20120512 -17.89591   137.0116        Grevillea striata   
62  NTAGFU0013    20120512 -17.89591   137.0116   Melaleuca stenostachya   
63  NTAGFU0013    20120

C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is tryin

      site_name  visit_date  latitude  longitude                 species  \
129  NTAGFU0029    20120615  -15.5381    135.409   Bauhinia cunninghamii   
130  NTAGFU0029    20120615  -15.5381    135.409          Corymbia bella   
131  NTAGFU0029    20120615  -15.5381    135.409                Denhamia   
132  NTAGFU0029    20120615  -15.5381    135.409   Eucalyptus microtheca   
133  NTAGFU0029    20120615  -15.5381    135.409    Eucalyptus tectifica   
134  NTAGFU0029    20120615  -15.5381    135.409       Grevillea striata   
135  NTAGFU0029    20120615  -15.5381    135.409    Petalostigma banksii   
136  NTAGFU0029    20120615  -15.5381    135.409    Terminalia canescens   
137  NTAGFU0029    20120615  -15.5381    135.409  Terminalia platyphylla   

      ba_m2ha  uid  datum                     geometry co_ef_nme  ...  \
129  0.300000   34  gda94  POINT (135.40900 -15.53810)      Tefe  ...   
130  0.250000   34  gda94  POINT (135.40900 -15.53810)      Tefe  ...   
131  0.100000   34  

C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is tryin

      site_name  visit_date  latitude  longitude             species  ba_m2ha  \
205  NTAMGD0005    20160429 -17.45585   133.4836    Acacia gonoclada  0.70000   
206  NTAMGD0005    20160429 -17.45585   133.4836     Corymbia setosa  2.95625   
207  NTAMGD0005    20160429 -17.45585   133.4836  Grevillea refracta  0.30000   

     uid  datum                     geometry co_ef_nme  ...  root_r2  tot_bm  \
205   51  gda94  POINT (133.48360 -17.45585)      Tefe  ...     0.57  3182.0   
206   51  gda94  POINT (133.48360 -17.45585)      Tefe  ...     0.57  3182.0   
207   51  gda94  POINT (133.48360 -17.45585)      Tefe  ...     0.57  3182.0   

      c_leaves    c_twigs      c_bark      c_wood  c_branches   c_stems  \
205   65.10000   67.90000   265.30000   863.10000   654.50000  1128.400   
206  274.93125  286.75625  1120.41875  3645.05625  2764.09375  4765.475   
207   27.90000   29.10000   113.70000   369.90000   280.50000   483.600   

      c_agb    c_roots  
205  1848.0   379.4000  
206

C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is tryin

In [59]:
df7

,site_name,visit_date,latitude,longitude,species,ba_m2ha,uid,datum,geometry,co_ef_nme,...,bio_r_kg1ha,bio_agb_kg1ha,c_l_kg1ha,c_t_kg1ha,c_b_kg1ha,c_w_kg1ha,c_br_kg1ha,c_s_kg1ha,c_r_kg1ha,c_agb_kg1ha
0,NTAARP0001,20160602,-13.55729,132.2701,Acacia mimula,1.050000,1,gda94,POINT (132.27010 -13.55729),Tefe,...,21520.226757,103818.027211,1941.533333,2023.688889,7421.322222,23579.2000,17928.777778,31000.522222,10544.911111,50870.833333
1,NTAARP0001,20160602,-13.55729,132.2701,Buchanania obovata,0.583333,1,gda94,POINT (132.27010 -13.55729),Tefe,...,21520.226757,103818.027211,1941.533333,2023.688889,7421.322222,23579.2000,17928.777778,31000.522222,10544.911111,50870.833333
2,NTAARP0001,20160602,-13.55729,132.2701,Erythrophleum chlorostachys,2.166667,1,gda94,POINT (132.27010 -13.55729),Erch,...,21520.226757,103818.027211,1941.533333,2023.688889,7421.322222,23579.2000,17928.777778,31000.522222,10544.911111,50870.833333
3,NTAARP0001,20160602,-13.55729,132.2701,Eucalyptus tetrodonta,14.833333,1,gda94,POINT (132.27010 -13.55729),Tefe,...,21520.226757,103818.027211,1941.533333,2023.688889,7421.322222,23579.2000,17928.777778,31000.522222,10544.911111,50870.833333
4,NTAARP0001,20160602,-13.55729,132.2701,Owenia vernicosa,0.200000,1,gda94,POINT (132.27010 -13.55729),Tefe,...,21520.226757,103818.027211,1941.533333,2023.688889,7421.322222,23579.2000,17928.777778,31000.522222,10544.911111,50870.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,NTTSTU0001,20140315,-16.63024,133.2712,Gyrocarpus americanus,0.100000,63,gda94,POINT (133.27120 -16.63024),Tefe,...,17641.116780,85927.210884,1483.220833,1547.015278,6044.523611,19664.6375,14911.951389,25709.161111,8644.147222,42104.333333
288,NTTSTU0001,20140315,-16.63024,133.2712,Macropteranthes kekwickii,3.187500,63,gda94,POINT (133.27120 -16.63024),Tefe,...,17641.116780,85927.210884,1483.220833,1547.015278,6044.523611,19664.6375,14911.951389,25709.161111,8644.147222,42104.333333
289,NTTSTU0001,20140315,-16.63024,133.2712,Melaleuca viridiflora,0.100000,63,gda94,POINT (133.27120 -16.63024),Tefe,...,17641.116780,85927.210884,1483.220833,1547.015278,6044.523611,19664.6375,14911.951389,25709.161111,8644.147222,42104.333333
290,NTTSTU0001,20140315,-16.63024,133.2712,Terminalia canescens,0.200000,63,gda94,POINT (133.27120 -16.63024),Tefe,...,17641.116780,85927.210884,1483.220833,1547.015278,6044.523611,19664.6375,14911.951389,25709.161111,8644.147222,42104.333333


In [60]:
df7.to_csv(os.path.join(output_dir, "kg1ha_tern_bio_c.csv"), index=False)
df7.to_csv(os.path.join(output_dir_ext, "kg1ha_tern_bio_c.csv"), index=False)

In [61]:
df7.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 292 entries, 0 to 291
Data columns (total 50 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   site_name      292 non-null    object  
 1   visit_date     292 non-null    int64   
 2   latitude       292 non-null    float64 
 3   longitude      292 non-null    float64 
 4   species        292 non-null    object  
 5   ba_m2ha        292 non-null    float64 
 6   uid            292 non-null    int64   
 7   datum          292 non-null    object  
 8   geometry       292 non-null    geometry
 9   co_ef_nme      292 non-null    object  
 10  leaves         292 non-null    float64 
 11  leaves_r2      292 non-null    float64 
 12  twigs          292 non-null    float64 
 13  bark           292 non-null    float64 
 14  bark_r2        292 non-null    float64 
 15  wood           292 non-null    float64 
 16  wood_r2        292 non-null    float64 
 17  branches       292 non-null

In [62]:
dict_ = {"site_name": "site",
        "visit_date": "date",
        "latitude": "lat_gda94",
        "longitude":"lon_gda94"}


# call rename () method
df7.rename(columns=dict_,
          inplace=True)

In [63]:
df_totals = df7[["uid", "site", "date", "lon_gda94", "lat_gda94", "bio_l_kg1ha", "bio_t_kg1ha", "bio_b_kg1ha", "bio_w_kg1ha",
                 "bio_br_kg1ha", "bio_s_kg1ha", "bio_r_kg1ha", "bio_agb_kg1ha", "c_l_kg1ha", "c_t_kg1ha",
                 "c_b_kg1ha", "c_w_kg1ha", "c_br_kg1ha", "c_s_kg1ha", "c_r_kg1ha", "c_agb_kg1ha"]]

In [64]:
# df_final[df_final["uid"] == 1]

In [65]:
df_totals.drop_duplicates(subset=["site"], inplace=True)
out_f_path = os.path.join(output_dir, "tern_c_bio_site_totals.csv")
df_totals.to_csv(out_f_path, index=False)
print("file exported: ", out_f_path)

file exported:  C:\Users\robot\projects\biomass\agb\20250122\tern_c_bio_site_totals.csv


C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
df_totals.drop_duplicates(subset=["site"], inplace=True)
out_f_path = os.path.join(output_dir_ext, "tern_c_bio_site_totals.csv")
df_totals.to_csv(out_f_path, index=False)
print("file exported: ", out_f_path)

file exported:  E:\biomass\agb\20250122\tern_c_bio_site_totals.csv


C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [67]:
df_totals = df7[["uid", "site", "date", "lon_gda94", "lat_gda94", "ba_m2ha", "bio_l_kg1ha", "bio_t_kg1ha", "bio_b_kg1ha", "bio_w_kg1ha",
                 "bio_br_kg1ha", "bio_s_kg1ha", "bio_r_kg1ha", "bio_agb_kg1ha", "c_l_kg1ha", "c_t_kg1ha",
                 "c_b_kg1ha", "c_w_kg1ha", "c_br_kg1ha", "c_s_kg1ha", "c_r_kg1ha", "c_agb_kg1ha"]]

In [68]:
df_totals.rename(columns={"ba_m2ha": "tot_ba_m2ha"}, inplace=True)

C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [69]:
df_totals.drop_duplicates(subset=["site"], inplace=True)
out_f_path = os.path.join(output_dir, "tern_c_bio_site_totals_w_ba.csv")
df_totals.to_csv(out_f_path, index=False)
print("file exported: ", out_f_path)

file exported:  C:\Users\robot\projects\biomass\agb\20250122\tern_c_bio_site_totals_w_ba.csv


C:\Users\robot\anaconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
